# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов, в 2 из них есть пропущенные значения, в таблице присутсвуют типы: int, float, object.

## Предобработка данных

### Обработка пропусков

In [2]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

 Пропущенные значения были обнаружены в столбцах `days_employed` и `total_income`.

In [3]:
data[(data['days_employed']).isna() & (data['days_employed']).isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропуски в `days_employed` и `total_income` совпадают.

In [4]:
data[(data['days_employed'].isna()) & (data['education'].str.lower() == 'среднее')]['education'].count()

1540

Более 2/3 строк с пропущенными значениями в столбцах `days_employed` и `total_income` также имеют значение 'среднее' в столбце `education`. Вероятно, люди со средним образованием имеют проблемы с поиском работы, а соответственно и низкий трудовой стаж, также люди со средним образованием, как правило, имеют ежемесячный доход ниже, чем у людей с высшим образованием. По этим причинам, скорее всего, в столбцах `days_employed` и `total_income` есть пропуски.

In [5]:
data[(data['dob_years'] == 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Также есть нули в столбце `dob_years`. Чтобы заполнить столбец `dob_years`, найдем средние значения возраста по столбцу `income_type`.

In [6]:
for unique_types in data['income_type'].unique():
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == unique_types), 'dob_years'] \
    = int(data[data['income_type'] == unique_types]['dob_years'].mean())

Когда появились все значения в столбце `dob_years`, можно заменить пропуски в `days_employed` и `total_income`.

In [7]:
for unique_dob in data['dob_years'].unique():
    data.loc[(data['days_employed'].isna()) & (data['dob_years'] == unique_dob), 'days_employed'] \
        = data[data['dob_years'] == unique_dob]['days_employed'].mean() 
    data.loc[(data['total_income'].isna()) & (data['dob_years'] == unique_dob), 'total_income'] \
        = data[data['dob_years'] == unique_dob]['total_income'].mean()
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбцы с пропущенными значениями данные заполнялись следующим образом: 
для каждого значения возраста был посчитан средний стаж работы в днях и средний ежемесячный доход, 
затем в каждую ячейку с пропуском в столбцах `days_employed` и `total_income` с соответствующим возрастом записывались средние значения.


In [8]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропущенных значений больше нет.

 Стоит посмотреть уникальные значения столбца `children`.

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видно, что есть 47 отрицательных значений -1. Так как их количество невелико, то можно заменить их на 1.

In [10]:
data.loc[data['children'] == -1, 'children'] = 1
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Отрицательных значений в столбце `children` больше нет.

В 2 столбцах были обнаружены пропущенные значения, которые, вероятно, были связаны с уровнем образования клиента. Эти пропуски были заменены на средние значения столбцов `days_employed` и `total_income` по возрасту клиента.

### Замена типа данных

Если смотреть на значения столбца `days_employed`, то можно заметить, что отрицательные значения не превышают по модулю 20000, а большинство положительных намного больше. Можно предположить, что значения с '-' означают дни, а значения с '+' означают часы.

In [11]:
data.loc[data['days_employed'] > 0, 'days_employed'] = data['days_employed'] / 24
data.loc[data['days_employed'] < 0, 'days_employed'] = -data['days_employed']
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Для изменения типа данных в столбцах `days_employed` и `total_income` использовался метод `astype()` с аргументом `'int'`, так как этот метод удобен для измениния типа данных на `'int'`.

### Обработка дубликатов

In [12]:
data.duplicated().sum()

54

Сначала найдено 54 дубликата. Но необходимо еще привести строки к единому регистру. 

In [13]:
data['education'] = data['education'].str.lower()
data.duplicated().sum()

71

Теперь дубликатов - 71.

In [14]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

Явных дубликатов больше нет. Вероятно, дубликаты появились в случае повторного обращения клиента в банк спустя время или в случае неправильной записи его личных данных.

В ходе обработки дубликатов был выбран метод `duplicated()`, потому что им удобно подсчитывать идентичные строки, в то время как `value_counts()` подсчитывал бы уникальные значения с их частотой, хотя в таблице все значения могут повторяться. 

### Лемматизация

Нелбходимо посмотреть все значения столбца `purpose`.

In [15]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Ключевые леммы здесь: недвижимость, свадьба, образование и автомобиль. Функция `lemma_func()` будет искать в значениях столбца `purpose` ключевые леммы и заменять ими значения столбца.

In [16]:
from pymystem3 import Mystem
m = Mystem()
def lemma_func(row):
    for string in row:
        lemmas = m.lemmatize(str(string))
        try:
            for word in lemmas:
                if word in lemmas:
                    if word == 'жилье':
                        return ('недвижимость')
                    elif word == 'недвижимость':
                        return ('недвижимость')
                    elif word == 'свадьба':
                        return ('свадьба')
                    elif word == 'образование':
                        return ('образование')
                    elif word == 'автомобиль':
                        return ('автомобиль')
        except:
            return ('Не найдено')
data['purpose'] = data.apply(lemma_func, axis = 1)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Из множества значений столбца `purpose` были выделены 4 ключевые леммы. Затем все значения в этом столбце заменялись на эти леммы.

### Категоризация данных

In [17]:
education = pd.Series(data = data['education'].unique(), index = data['education_id'].unique())
family_status = pd.Series(data = data['family_status'].unique(), index = data['family_status_id'].unique())
display(education)
display(family_status)

0                 высшее
1                среднее
2    неоконченное высшее
3              начальное
4         ученая степень
dtype: object

0          женат / замужем
1         гражданский брак
2           вдовец / вдова
3                в разводе
4    Не женат / не замужем
dtype: object

**Вывод**

В таблице категоризацию данных можно провести серди 2 пар столбцов: `education`, `education_id` и `family_status`, `family_status_id`. Каждому значению из столбца с id соответствует конкретное значение из другого столбца.

## Основные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для наглядности необходимо создать сводную таблицу `data_pivot_children`.

In [18]:
data_pivot_children = pd.pivot_table(data, index=['children'], columns=['debt'], values=['dob_years'],\
        aggfunc=['count'], fill_value=0)
display(data_pivot_children)

count      
         dob_years      
debt             0     1
children                
0            13028  1063
1             4410   445
2             1858   194
3              303    27
4               37     4
5                9     0
20              68     8

Больше всего клиентов, у которых нет детей. У них наибольшие показатели и по наличию долгов, и по их отсутствию. Чтобы понять, как количество детей влияет на возврат кредитов, нужно для каждого значения в столбце `children` найти отношение количества клиентов с долгом к общему количеству клиентов. 

In [19]:
def percent(data_pivot):
    percentage = data_pivot.values.tolist()
    col = data_pivot.index.tolist()
    for num in range(len(percentage)):
        display(f'{col[num]}: {percentage[num][1] /(percentage[num][0] + percentage[num][1]) * 100 :.2f}%')    
percent(data_pivot_children)

'0: 7.54%'

'1: 9.17%'

'2: 9.45%'

'3: 8.18%'

'4: 9.76%'

'5: 0.00%'

'20: 10.53%'

Видно, что с ростом количества детей процент клиентов с долгом увеличивается, кроме клиентов с 5 детьми.

Проанализировав сводную таблицу, можно сделать вывод, что чем больше детей, тем больше процент невозврата кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
data_pivot_family = pd.pivot_table(data, index=['family_status'], columns=['debt'], values=['dob_years'],\
        aggfunc=['count'], fill_value=0)
display(data_pivot_family)

count     
                      dob_years     
debt                          0    1
family_status                       
Не женат / не замужем      2536  274
в разводе                  1110   85
вдовец / вдова              896   63
гражданский брак           3763  388
женат / замужем           11408  931

In [21]:
percent(data_pivot_family)

'Не женат / не замужем: 9.75%'

'в разводе: 7.11%'

'вдовец / вдова: 6.57%'

'гражданский брак: 9.35%'

'женат / замужем: 7.55%'

Из вывода функции видно, что наличие долга не зависит от того, состоит человек в отношениях или нет. Меньше всего долгов у вдовцов и вдов, больше всего у не женатых и не замужних.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сначала необходимо сгруппировать значения столбца `total_income` по уровню дохода.

In [22]:
def total_income(total):
    income = total['total_income']
    if income < 50000:
        return 'низкий доход'
    elif income >= 50000 and income < 150000:
        return 'средний доход'
    elif income > 150000:
        return 'высокий доход'                
data['income_grade'] = data.apply(total_income, axis = 1)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_grade
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий доход
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,высокий доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость,высокий доход
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль,высокий доход
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,средний доход
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль,высокий доход


In [23]:
data_pivot_income = pd.pivot_table(data, index=['income_grade'], columns=['debt'], values=['dob_years'],\
        aggfunc=['count'], fill_value=0)
display(data_pivot_income)

count     
              dob_years     
debt                  0    1
income_grade                
высокий доход     10178  877
низкий доход        349   23
средний доход      9186  841

In [24]:
percent(data_pivot_income)

'высокий доход: 7.93%'

'низкий доход: 6.18%'

'средний доход: 8.39%'

Из вывода видно, что люди с низким доходом возвращают кредит в срок чаще, чем люди с большим доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_pivot_purpose = pd.pivot_table(data, index=['purpose'], columns=['debt'], values=['dob_years'],\
        aggfunc=['count'], fill_value=0)
display(data_pivot_purpose)

count     
             dob_years     
debt                 0    1
purpose                    
автомобиль        3903  403
недвижимость     10029  782
образование       3643  370
свадьба           2138  186

In [26]:
percent(data_pivot_purpose)

'автомобиль: 9.36%'

'недвижимость: 7.23%'

'образование: 9.22%'

'свадьба: 8.00%'

Клиенты, которые берут кредит на недвижимость или свадьбу возвращают их чаще, чем кредиты, взятые для других целей. 

## Общий вывод

Анализ таблицы показал, как семейное положение и количество детей клиента на факт погашения кредита в срок. 
- Чем больше детей, тем больше процент невозврата кредита,
- Семейное положение клиента не влияет на возврат кредита в срок,
- Клиенты с низким доходом возвращают кредит в срок чаще, чем люди с большим доходом,
- Кредиты, взятые на свадьбу или недвижимость возвращаются в срок чаще.